In [6]:
import os
import cv2
import uuid

In [3]:
base_path = "/home/chess/Documents/data/ML_raw_data/numberplate/plate_detection/data"
images_path = os.path.join(base_path, "images")
labels_path = os.path.join(base_path, "labels")
cropped_output_path = os.path.join(base_path, "cropped")

In [4]:
if not os.path.exists(cropped_output_path):
    os.mkdir(cropped_output_path)

## Show Image and Plot labels 

In [5]:
for image in os.listdir(images_path):
    image_name = ".".join(image.split(".")[:-1])
    image_path = os.path.join(images_path, image)
    label_path = os.path.join(labels_path, f"{image_name}.txt")
    
    if not os.path.exists(label_path):
        print(f"Image {image} label does not exist")
    
    # Read image and plot
    img = cv2.imread(image_path)
    dh, dw, _ = img.shape

    fl = open(label_path, 'r')
    data = fl.readlines()
    fl.close()

    for dt in data:
        _, x, y, w, h = map(float, dt.split(' '))

        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)
        
        if l < 0:
            l = 0
        if r > dw - 1:
            r = dw - 1
        if t < 0:
            t = 0
        if b > dh - 1:
            b = dh - 1

        cv2.rectangle(img, (l, t), (r, b), (0, 0, 255), 1)

    cv2.imshow("plotted image", img)
    key = cv2.waitKey(0)
    if key == ord("q"):
        break

cv2.destroyAllWindows()

### Crop images based on labels

In [7]:
for image in os.listdir(images_path):
    image_name = ".".join(image.split(".")[:-1])
    image_path = os.path.join(images_path, image)
    label_path = os.path.join(labels_path, f"{image_name}.txt")
    
    if not os.path.exists(label_path):
        print(f"Image {image} label does not exist")
    
    # Read image and plot
    img = cv2.imread(image_path)
    dh, dw, _ = img.shape

    fl = open(label_path, 'r')
    data = fl.readlines()
    fl.close()

    for dt in data:
        _, x, y, w, h = map(float, dt.split(' '))

        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)
        
        if l < 0:
            l = 0
        if r > dw - 1:
            r = dw - 1
        if t < 0:
            t = 0
        if b > dh - 1:
            b = dh - 1

        cropped_image = img[t + 5 : b + 5, l + 5 : r + 5]
        idx = uuid.uuid4()
        cropped_image_path = os.path.join(cropped_output_path, f"{idx}.jpg")
        cv2.imwrite(cropped_image_path, cropped_image)
    

## convert yolo bounding box into label files

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import imutils
import os
import shutil

In [ ]:
model = YOLO("./models/plate_segment.pt")

In [ ]:
num_plates_path = "./data/images/"
output_path = "/home/chess/Documents/data/temp/detect"
for image in os.listdir(num_plates_path):
    image_path = os.path.join(num_plates_path, image)
    frame = cv2.imread(image_path)
    if frame is None:
        continue
    frame = imutils.resize(frame, width=1200)
    dh, dw, _ = frame.shape
    results = model.predict(frame, verbose=False)

    result = results[0]
    lines = []
    for box in result.boxes:
        cords = box.xyxy[0].tolist()
        # cords = [round(x) for x in cords]
        x_min, y_min, x_max, y_max = cords
        x_min, y_min, x_max, y_max = int(x_min) - 1, int(y_min) -1, int(x_max) +1, int(y_max) + 1

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
        
        # converts the bbox into the labels
        x = (x_min / dw) + ((x_max - x_min) / (2 * dw))
        y = (y_min / dh) + ((y_max - y_min) / (2 * dh))
        w = (x_max - x_min) / dw
        h = (y_max - y_min) / dh

        line = f"0 {x} {y} {w} {h}"
        lines.append(line)
    
    # print("---------------------------------")
    cv2.imshow("frame", frame)
    
    curr_id = image.split(".")[0]
    
    output_image_path = os.path.join(output_path, "images", image)
    label_path = os.path.join(output_path, "labels", f"{curr_id}.txt")

    shutil.copy2(image_path, output_image_path)
    with open(label_path, "w") as f:
        f.writelines(lines)

    key = cv2.waitKey(1)
    if key == ord("q"):
        break


cv2.destroyAllWindows()